# Workshop #2 :
## Reccobeats API - EDA

------------------------------------------------------------

https://reccobeats.com/docs/apis/extract-audio-features

In [1]:
import os
import sys
import pandas as pd
import logging


sys.path.append(os.path.abspath('../'))
from src.params import Params
from src.client import DatabaseClient
from src.logging_config import setup_logging

In [3]:
setup_logging()

In [ ]:
df = pd.read_csv("../data/external/reccobeats_features.csv")

In [6]:
df.shape

(60, 10)

In [14]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 60 entries, 0 to 59
Data columns (total 10 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   nominee           60 non-null     object 
 1   acousticness      60 non-null     float64
 2   danceability      60 non-null     float64
 3   energy            60 non-null     float64
 4   instrumentalness  60 non-null     float64
 5   liveness          60 non-null     float64
 6   loudness          60 non-null     float64
 7   speechiness       60 non-null     float64
 8   tempo             60 non-null     float64
 9   valence           60 non-null     float64
dtypes: float64(9), object(1)
memory usage: 4.8+ KB


**acousticness** `float`

Acousticness refers to how much of a song or piece of music is made up of natural, organic sounds rather than synthetic or electronic elements. In other words, it's a measure of how "acoustic" a piece of music sounds. A confidence measure from 0.0 to 1.0, greater value represents higher confidence the track is acoustic.


**danceability** `float`

Danceability is a measure of how suitable a song is for dancing, ranging from 0 to 1. A score of 0 means the song is not danceable at all, while a score of 1 indicates it is highly danceable. This score takes into account factors like tempo, rhythm, beat consistency, and energy, with higher scores indicating stronger, more rhythmically engaging tracks.


**energy** `float`

Energy in music refers to the intensity and liveliness of a track, with a range from 0 to 1. A score of 0 indicates a very calm, relaxed, or low-energy song, while a score of 1 represents a high-energy, intense track. It’s influenced by elements like tempo, loudness, and the overall drive or excitement in the music.


**instrumentalness** `float`

Predicts whether a track contains no vocals. “Ooh” and “aah” sounds are treated as instrumental in this context. Rap or spoken word tracks are clearly “vocal”. The closer the instrumentalness value is to 1.0, the greater likelihood the track contains no vocal content. Values above 0.5 are intended to represent instrumental tracks, but confidence is higher as the value approaches 1.0.


**liveness** `float`

Detects the presence of an audience in the recording. Higher liveness values represent an increased probability that the track was performed live. A value above 0.8 provides strong likelihood that the track is live.


**loudness** `float`

The overall loudness of a track in decibels (dB). Loudness values are averaged across the entire track and are useful for comparing relative loudness of tracks. Loudness is the quality of a sound that is the primary psychological correlate of physical strength (amplitude). Values typical range between -60 and 0 db.


**speechiness** `float`

Speechiness detects the presence of spoken words in a track. The more exclusively speech-like the recording (e.g. talk show, audio book, poetry), the closer to 1.0 the attribute value. Values above 0.66 describe tracks that are probably made entirely of spoken words. Values between 0.33 and 0.66 describe tracks that may contain both music and speech, either in sections or layered, including such cases as rap music. Values below 0.33 most likely represent music and other non-speech-like tracks.


**tempo** `float`

The overall estimated tempo of a track in beats per minute (BPM). Values typical range between 0 and 250


**valence** `float`

Valence in music measures the emotional tone or mood of a track, with a range from 0 to 1. A score of 0 indicates a song with a more negative, sad, or dark feeling, while a score of 1 represents a more positive, happy, or uplifting mood. Tracks with a high valence tend to feel joyful or energetic, while those with a low valence may evoke feelings of melancholy or sadness.

In [5]:
df.isnull().sum()

nominee             0
acousticness        0
danceability        0
energy              0
instrumentalness    0
liveness            0
loudness            0
speechiness         0
tempo               0
valence             0
dtype: int64

In [9]:
# Find duplicated rows
duplicated_rows = df[df.duplicated(keep=False)]  # Keep all duplicates (including the first occurrence)

duplicated_rows


,nominee,acousticness,danceability,energy,instrumentalness,liveness,loudness,speechiness,tempo,valence
0,Bad Guy,0.6986,0.6329,0.1706,0.0383,0.1547,-22.5960,0.2376,103.9933,0.3587
1,Bad Guy,0.6986,0.6329,0.1706,0.0383,0.1547,-22.5960,0.2376,103.9933,0.3587
2,This Is America,0.9028,0.5238,0.2993,0.0077,0.2014,-13.8596,0.1889,91.1617,0.6618
3,This Is America,0.9028,0.5238,0.2993,0.0077,0.2014,-13.8596,0.1889,91.1617,0.6618
6,Hello,0.5035,0.2270,0.2031,0.0550,0.1949,-25.8341,0.0906,70.9526,0.0736
7,Hello,0.5035,0.2270,0.2031,0.0550,0.1949,-25.8341,0.0906,70.9526,0.0736
13,Rolling In The Deep,0.5311,0.6745,0.1801,0.0073,0.1093,-18.6429,0.0752,111.9134,0.6375
14,Rolling In The Deep,0.5311,0.6745,0.1801,0.0073,0.1093,-18.6429,0.0752,111.9134,0.6375
15,Need You Now,0.5975,0.4404,0.2154,0.8670,0.1125,-17.8910,0.0389,100.7200,0.1474
16,Need You Now,0.5975,0.4404,0.2154,0.8670,0.1125,-17.8910,0.0389,100.7200,0.1474


In [10]:
df1 = df.drop_duplicates(keep='first')

In [15]:
df1.describe()

,acousticness,danceability,energy,instrumentalness,liveness,loudness,speechiness,tempo,valence
count,47.000000,47.000000,47.000000,47.000000,47.000000,47.000000,47.000000,47.000000,47.000000
mean,0.612936,0.529470,0.286289,0.270621,0.139551,-16.643489,0.090064,105.319102,0.367077
std,0.290106,0.178532,0.190017,0.345548,0.040955,5.011661,0.078100,13.065411,0.233941
min,0.009100,0.214700,0.029200,0.000000,0.094300,-26.162500,0.033200,70.952600,0.063600
25%,0.335450,0.401600,0.134050,0.009300,0.113050,-20.639300,0.040350,98.201200,0.196500
50%,0.705200,0.513600,0.230900,0.038300,0.123400,-16.392500,0.053400,103.584500,0.323100
75%,0.846350,0.670950,0.430800,0.638850,0.146550,-12.523300,0.090000,113.940900,0.579450
max,0.972200,0.872600,0.744400,0.867000,0.285200,-6.322100,0.360200,152.098300,0.895100
